<a href="https://colab.research.google.com/github/grace-arina/Diagnosing-Pneumonia_CNN/blob/Olgert/Final_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pneumonia

In [1]:
#Mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Importing necessary dependencies 
import pandas as pd
import numpy as np
np.random.seed(123)
import datetime

import scipy
import os, shutil
from PIL import Image
from scipy import ndimage


import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, SeparableConv2D , BatchNormalization,Dropout
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Rescaling, RandomFlip, RandomRotation, RandomTranslation, RandomZoom
from keras.callbacks import EarlyStopping, ModelCheckpoint, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')


import warnings
warnings.filterwarnings('ignore')

%load_ext tensorboard

## Setting up models with Tensorflow 2.5
Version capped at Tensorflow 2.5 to use local GPU

In [3]:
#Loading files and recaling images
idg = ImageDataGenerator(rescale=1./255)

train_set = idg.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/chest_xray/train',
                                                 target_size=(150, 150),
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 color_mode='grayscale')

val_set = idg.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/chest_xray/val',
                                          target_size=(150, 150),
                                          batch_size=32,
                                          class_mode='binary',
                                          color_mode='grayscale')

test_set = idg.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/chest_xray/test',
                                            target_size=(150, 150),
                                            batch_size=32,
                                            class_mode='binary',
                                            color_mode='grayscale')

Found 5022 images belonging to 2 classes.
Found 210 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Setting up Baseline Model

In [8]:
# Setting paramaters on early stopping
earlystop = EarlyStopping(monitor='val_loss',
                          patience=20,
                          verbose=1,
                          mode='min',
                          restore_best_weights=True)

#If recreating notebook, use the below 3 lines of code to save a checkpoint for your models, where your logs should be saved and the Tensorboard callback to save your models in Tensorboard

# mod_checkpt = ModelCheckpoint(filepath ="/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/Models/my_model1.h5",monitor= "val_loss", save_best_only=True )
# log_dir = "/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/Models/"
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [11]:
#Setting up a baseline model
model1 = models.Sequential()

#1st Layer
model1.add(layers.Conv2D(32, 7, input_shape=(150,150,1), padding='same',activation='relu'))
model1.add(layers.MaxPooling2D(2))

#2nd layer
model1.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model1.add(layers.MaxPooling2D(2))

#Flattening data into 1 vector
model1.add(layers.Flatten())
model1.add(layers.Dense(64, activation='relu'))
model1.add(layers.Dense(32, activation='relu'))
model1.add(layers.Dense(1, activation='sigmoid'))

#Compiling model with adam optemizer, binary_crossentropy since this is binairy classification and setting metrics to accuracy and recall
model1.compile(optimizer="adam", loss="binary_crossentropy", metrics=['acc', tf.metrics.Recall()])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 150, 150, 32)      1600      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 87616)             0         
                                                                 
 dense_3 (Dense)             (None, 64)               

In [14]:
#Fitting model, with 100 steps and 30 epochs and using early stop when our model stops improving
history1 = model1.fit(train_set,
                    validation_data=val_set,
                    steps_per_epoch=100,
                    epochs=30,
                    callbacks=[earlystop])

Epoch 1/30
100/100 [==============================] - 49s 480ms/step - loss: 0.2441 - acc: 0.9006 - recall_1: 0.9575 - val_loss: 0.1542 - val_acc: 0.9429 - val_recall_1: 0.9276
Epoch 2/30
100/100 [==============================] - 48s 477ms/step - loss: 0.1079 - acc: 0.9600 - recall_1: 0.9759 - val_loss: 0.0965 - val_acc: 0.9714 - val_recall_1: 0.9671
Epoch 3/30
100/100 [==============================] - 49s 488ms/step - loss: 0.0983 - acc: 0.9669 - recall_1: 0.9775 - val_loss: 0.1228 - val_acc: 0.9429 - val_recall_1: 0.9868
Epoch 4/30
100/100 [==============================] - 49s 484ms/step - loss: 0.0815 - acc: 0.9694 - recall_1: 0.9805 - val_loss: 0.1253 - val_acc: 0.9476 - val_recall_1: 0.9868
Epoch 5/30
100/100 [==============================] - 45s 446ms/step - loss: 0.0702 - acc: 0.9716 - recall_1: 0.9833 - val_loss: 0.1039 - val_acc: 0.9571 - val_recall_1: 0.9474
Epoch 6/30
100/100 [==============================] - 45s 449ms/step - loss: 0.0587 - acc: 0.9791 - recall_1: 0.986

In [15]:
#Evaluating model on test data
model1.evaluate(test_set)

20/20 [==============================] - 94s 5s/step - loss: 4.0147 - acc: 0.7308 - recall_1: 0.9949


[4.014666557312012, 0.7307692170143127, 0.9948717951774597]

## Second Model With Image Augmentation

In [16]:
# This is the same as the best previous model with image augmentation
augment_gen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=.2,
                                   height_shift_range=.2,
                                   shear_range=.2,
                                   zoom_range=.2,
                                   brightness_range=[.5, 1.5])

augment_set = augment_gen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/chest_xray/train',
                                                 target_size=(150, 150),
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 color_mode='grayscale')

augmented = models.Sequential()

augmented.add(layers.Conv2D(32, 7, padding='valid', input_shape=(150,150,1), activation='relu'))
augmented.add(layers.MaxPooling2D(2))

augmented.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
augmented.add(layers.MaxPooling2D(2))

augmented.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
augmented.add(layers.MaxPooling2D(3))

augmented.add(layers.Flatten())
augmented.add(layers.Dense(64, activation='relu'))
augmented.add(layers.Dense(1, activation='sigmoid'))

augmented.compile(optimizer="adam", loss="binary_crossentropy", metrics=['acc', tf.metrics.Recall()])

augmented.summary()

Found 5022 images belonging to 2 classes.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 144, 144, 32)      1600      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 72, 72, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 36, 36, 64)        36928     
                                                                 
 max_pooling

In [17]:
historyaug = augmented.fit(augment_set,
                    validation_data=val_set,
                    steps_per_epoch=100,
                    epochs=30,
                    callbacks=[earlystop])

Epoch 1/30
100/100 [==============================] - 61s 580ms/step - loss: 0.5348 - acc: 0.7419 - recall_2: 0.9395 - val_loss: 0.4802 - val_acc: 0.7429 - val_recall_2: 0.9539
Epoch 2/30
100/100 [==============================] - 60s 599ms/step - loss: 0.4535 - acc: 0.7649 - recall_2: 0.8573 - val_loss: 0.4251 - val_acc: 0.7381 - val_recall_2: 0.9934
Epoch 3/30
100/100 [==============================] - 57s 569ms/step - loss: 0.4131 - acc: 0.7919 - recall_2: 0.8869 - val_loss: 0.3370 - val_acc: 0.7952 - val_recall_2: 0.9013
Epoch 4/30
100/100 [==============================] - 56s 564ms/step - loss: 0.3737 - acc: 0.8184 - recall_2: 0.9047 - val_loss: 0.3180 - val_acc: 0.8619 - val_recall_2: 0.8553
Epoch 5/30
100/100 [==============================] - 59s 588ms/step - loss: 0.3567 - acc: 0.8469 - recall_2: 0.9099 - val_loss: 0.3050 - val_acc: 0.8333 - val_recall_2: 0.8882
Epoch 6/30
100/100 [==============================] - 59s 589ms/step - loss: 0.3057 - acc: 0.8684 - recall_2: 0.921

In [18]:
augmented.evaluate(test_set)

20/20 [==============================] - 7s 360ms/step - loss: 0.4662 - acc: 0.8317 - recall_2: 0.9282


[0.46619439125061035, 0.8317307829856873, 0.928205132484436]

## Final Model with Tensorflow 2.5

In [19]:
#Final Model

long_run = models.Sequential()

long_run.add(layers.Conv2D(32, 7, padding='valid', input_shape=(150,150,1), activation='relu'))
long_run.add(layers.MaxPooling2D(2))

long_run.add(layers.Conv2D(128, 3, padding='valid', activation='relu'))
long_run.add(layers.MaxPooling2D(2))

long_run.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
long_run.add(layers.MaxPooling2D(2))

long_run.add(layers.Conv2D(64, 2, padding='same', activation='relu'))
long_run.add(layers.MaxPooling2D(2))

long_run.add(layers.Flatten())
long_run.add(layers.Dense(64, activation='relu'))
long_run.add(layers.Dense(64, activation='relu'))
long_run.add(layers.Dense(1, activation='sigmoid'))

long_run.compile(optimizer="adam", loss="binary_crossentropy", metrics=['acc', tf.metrics.Recall()])
long_run.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 144, 144, 32)      1600      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 72, 72, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 70, 70, 128)       36992     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 35, 35, 128)      0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 35, 35, 64)        73792     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 17, 17, 64)      

In [ ]:
history_long_run = long_run.fit(augment_set,
                    validation_data=val_set,
                    epochs=200,
                    callbacks=[earlystop])

Epoch 1/200
157/157 [==============================] - 91s 579ms/step - loss: 0.5061 - acc: 0.7429 - recall_3: 0.9601 - val_loss: 0.4461 - val_acc: 0.7238 - val_recall_3: 1.0000
Epoch 2/200
157/157 [==============================] - 89s 566ms/step - loss: 0.4457 - acc: 0.7662 - recall_3: 0.9185 - val_loss: 0.4768 - val_acc: 0.7429 - val_recall_3: 1.0000
Epoch 3/200
157/157 [==============================] - 94s 595ms/step - loss: 0.3813 - acc: 0.8160 - recall_3: 0.8906 - val_loss: 0.3543 - val_acc: 0.8333 - val_recall_3: 0.8618
Epoch 4/200
157/157 [==============================] - 90s 572ms/step - loss: 0.3422 - acc: 0.8453 - recall_3: 0.8896 - val_loss: 0.3639 - val_acc: 0.7667 - val_recall_3: 0.9868
Epoch 5/200
157/157 [==============================] - 93s 590ms/step - loss: 0.3253 - acc: 0.8536 - recall_3: 0.8979 - val_loss: 0.2880 - val_acc: 0.8857 - val_recall_3: 0.9079
Epoch 6/200
157/157 [==============================] - 93s 589ms/step - loss: 0.3021 - acc: 0.8684 - recall_3:

In [ ]:
long_run.evaluate(test_set)

# Models with updated Tensor 2.8

In [ ]:
#Similar to above, setting image size and batch size
image_size = (150,150)
batch_size = 50

#Loading images from directory
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/chest_xray/train",
    image_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale")

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/chest_xray/test",
    image_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale")

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/chest_xray/val",
    image_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale")


In [ ]:
#Checking Class names
class_names = train_ds.class_names
print(class_names)

In [ ]:
#Checking size of imagees and labels
for image_batch, labels_batch in train_ds.take(1):
  print(image_batch.shape)
  print(f"Labels for train ds: {labels_batch}")
  break

In [ ]:
#Pulls labels from test_ds
test_labels= np.concatenate([y for x, y in test_ds], axis=0)

In [ ]:
#Creating function to evaluate our models

def model_eval(model):#input current model, train data and test data
  results_train = model.evaluate(train_ds) #evaluates train input
  results_test = model.evaluate(test_ds) #evaluates test input
  pred = model.predict(test_ds)
  preds = np.where(pred>0.5, 1,0)#if probability is less than .5 assign 0 if greater than 0 assign 1
  cm  = confusion_matrix(test_labels, preds, normalize="true")
  return print(f'Train Loss: {results_train[0]}, Train Accuracy: {results_train[1]} \n Test Loss: {results_test[0]}, Test Accuracy: {results_test[1]}.')

In [ ]:
#Setting autotune to pre-fetch train and validation data
tpain = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=tpain)
val_ds = val_ds.prefetch(buffer_size=tpain)

# Creating 1st model with SeparableConv2D

In [ ]:

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = layers.Rescaling(1./255)(inputs)
    x = layers.RandomRotation(0.6)(x)
    x = layers.RandomZoom(0.2)(x)

    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)    
    x = MaxPooling2D((2,2), name='pool1')(x)

#Introducing first SeparableConv2D and BatchNormalization to our modeling
    x = layers.SeparableConv2D(64, 3, padding="same", activation='relu')(x)
    x = layers.BatchNormalization()(x) 
    x = MaxPooling2D((2,2), name='pool2')(x)

    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)


    x = Flatten()(x)   
    x = Dense(256, activation='relu')(x)   
    x = Dense(128, activation='relu')(x)

    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model3 = make_model(input_shape=(image_size) + (1,), num_classes=2)
#Showing the models steps
keras.utils.plot_model(model3, show_shapes=True)

In [ ]:
model3.summary()

In [ ]:
#Model checkpoint monitor val_loss and save best model
#Setting log directory and using tensoboard callback so our data can be uploaded to Tensorboard 
#Similar to above, uncoment the below rows to save checkpoints and load data to TensorBoard

# mod_checkpt = ModelCheckpoint(filepath ="/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/Models/my_model3.h5",monitor= "val_loss", save_best_only=True )
#log_dir = "/content/drive/MyDrive/Colab Notebooks/Diagnosing-Pneumonia_CNN/Models/logs fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Using early stop to stop model after 10 epochs of no improvenment
early_stop = EarlyStopping(monitor= "val_loss", mode="min", patience = 10, restore_best_weights=True)
lr_rate = ReduceLROnPlateau(monitor="val_loss", patience=5, factor=.000000001)
callbacks = [early_stop, lr_rate]

In [ ]:
model3.compile(optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.metrics.Recall()])

result3 = model3.fit(train_ds, epochs=500, batch_size=50, validation_data=val_ds, shuffle=True, callbacks=callbacks)

In [ ]:
model3_eval = model_eval(model3)

## Second model with SeparableConv2D

In [ ]:

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = layers.Rescaling(1./255)(inputs)
    x = layers.RandomRotation(0.6)(x)
    x = layers.RandomZoom(0.2)(x)

    x = Conv2D(32, 3, activation='relu', padding='same')(x)    
    x = MaxPooling2D((2,2), name='pool1')(x)

#Introducing first SeparableConv2D and BatchNormalization to our modeling
    x = layers.SeparableConv2D(64, 3, padding="same", activation='relu')(x)
    x = layers.BatchNormalization()(x) 
    x = MaxPooling2D((2,2), name='pool2')(x)

    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)

    x = layers.SeparableConv2D(64, 3, padding="same", activation='relu')(x)
    x = layers.BatchNormalization()(x) 
    x = MaxPooling2D((2,2), name='pool2')(x)

    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)

    x = Flatten()(x)   
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)    
    x = Dense(128, activation='relu')(x)

    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model4 = make_model(input_shape=(image_size) + (1,), num_classes=2)
#Showing the models steps
keras.utils.plot_model(model4, show_shapes=True)

In [ ]:
model4.summary()

In [ ]:
model4.compile(optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.metrics.Recall()])

result4 = model4.fit(train_ds, epochs=500, batch_size=50, validation_data=val_ds, shuffle=True, callbacks=callbacks)

In [ ]:
model4_eval = model_eval(model4)

## Complex model with SeparableConv2D, BatchNormalization, SeparableConv2D loop with more sizes and residuals

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = layers.RandomRotation(0.6)(x)
    x = layers.RandomZoom(0.3)(x)


    x = layers.Rescaling(1./255)(inputs)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)

    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)


    previous_block_activation = x  # Set aside residual
    for size in [128, 256, 512]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation)
        
        x = layers.add([x, residual])  # Add back residual

    x = Flatten()(x)   
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x) 
    x = Dense(64, activation='relu')(x)   

    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model5 = make_model(input_shape=(image_size) + (1,), num_classes=2)
keras.utils.plot_model(model5, show_shapes=True)

In [ ]:
model5.summary()

In [ ]:
model5.compile(optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.metrics.Recall()])

result1 = model5.fit(train_ds, epochs=500, batch_size=50, validation_data=val_ds, shuffle=True, callbacks=callbacks)

Adding SeparableConv2D is not improving our models, its adding paramaters with no gains, reverting to using Conv2d to keep paramaters low and improve processing speed

## Conv2D with Tensor 2.8

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = layers.Rescaling(1./255)(inputs)
    x = layers.RandomRotation(0.6)(x)
    x = layers.RandomZoom(0.2)(x)

    x = Conv2D(32, 3, activation='relu', padding='same')(x)    
    x = MaxPooling2D((2,2), name='pool1')(x)

    x = Conv2D(64, 3, activation='relu', padding='same')(x)    
    x = MaxPooling2D((2,2), name='pool2')(x)    

    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)

    x = Conv2D(256, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)

    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), name='pool5')(x)    

    x = Flatten()(x)   
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)    
    x = Dense(128, activation='relu')(x)

    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model7 = make_model(input_shape=(image_size) + (1,), num_classes=2)
keras.utils.plot_model(model7, show_shapes=True)

In [ ]:
model7.summary()

In [ ]:
model7.compile(optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.metrics.Recall()])

result_7 = model7.fit(train_ds, epochs=500, batch_size=50, validation_data=val_ds, shuffle=True, callbacks=callbacks)

In [ ]:
model7_eval = model_eval(model7)

### To see a more iterative process please use the notebooks in "modeling-notebooks"

In conclusion the model that performed best was the 4 Layer Conv2D nerural network. It produced the best results with low paramaters which will make our model faster.